version 3.1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from torch.utils.data import TensorDataset, DataLoader


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load original dataset
df = pd.read_csv("framinghamdataset.csv")

# Features and labels
X = df.drop(columns=['TenYearCHD'])
y = df['TenYearCHD']

# Scaling data (must be consistent for server and clients)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset: 1/3 for global testing, 2/3 for client training
X_clients, X_global, y_clients, y_global = train_test_split(
    X_scaled, y, test_size=1/3, random_state=42, stratify=y
)

# Apply SMOTE only on the client training set (2/3 portion)
smote = SMOTE(random_state=42)
X_clients_res, y_clients_res = smote.fit_resample(X_clients, y_clients)

# Save data for Global server evaluation
np.save('X_global.npy', X_global)
np.save('y_global.npy', y_global)

# Save data for Client-side simulations
np.save('X_clients.npy', X_clients_res)
np.save('y_clients.npy', y_clients_res)

print("✅ Data successfully divided:")
print(f"Global evaluation data: {X_global.shape}")
print(f"Client training data (after SMOTE): {X_clients_res.shape}")


In [ ]:
X_global = np.load('X_global.npy')
y_global = np.load('y_global.npy')


In [3]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from typing import List, Optional, Tuple
from flwr.common import Parameters, FitRes, EvaluateRes, ndarrays_to_parameters

# ✅ Define the Global Model
class GlobalModel(nn.Module):
    def __init__(self, input_size):
        super(GlobalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.7),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# ✅ Load Data (Only Needed for Input Size)
X_clients = np.load('X_clients.npy')
input_size = X_clients.shape[1]

# ✅ Initialize the Global Model
global_model = GlobalModel(input_size)
global_parameters = [val.cpu().detach().numpy() for val in global_model.parameters()]

# ✅ Custom FedProx Strategy with Accuracy Calculation
class CustomFedProx(fl.server.strategy.FedAvg):
    def __init__(self, initial_parameters):
        super().__init__()
        self.initial_parameters = initial_parameters

    def initialize_parameters(self, client_manager):
        return ndarrays_to_parameters(self.initial_parameters)

    def aggregate_evaluate(
    self,
    rnd: int,
    results: List[Tuple[fl.common.Parameters, EvaluateRes]],
    failures: List[BaseException],
    ) -> Tuple[float, dict]:
        """Aggregate evaluation results and print global accuracy."""
        
        if not results:
            print(f"⚠️ Round {rnd}: No evaluation results received!")
            return 0.0, {"accuracy": 0.0}  # ✅ Return a valid tuple

        accuracies = [r.metrics["accuracy"] for _, r in results]
        global_accuracy = sum(accuracies) / len(accuracies)

        print(f"📊 Round {rnd}: Global Model Accuracy: {global_accuracy:.4f}")

        


# ✅ Initialize Strategy with Global Model Parameters
strategy = CustomFedProx(initial_parameters=global_parameters)

# ✅ Start Federated Learning Server (Runs for 30 Rounds)
fl.server.start_server(
    server_address="0.0.0.0:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=10)  # Adjust the number of rounds
)

# ✅ Evaluate the Final Global Model
global_model.eval()
print("🎯 Final Global Model Evaluation Completed!")


FileNotFoundError: [Errno 2] No such file or directory: 'X_clients.npy'

version 3.1.1

In [4]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from typing import List, Optional, Tuple
from flwr.common import Parameters, EvaluateRes, ndarrays_to_parameters


# ✅ Define the Global Model
class GlobalModel(nn.Module):
    def __init__(self, input_size):
        super(GlobalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# ✅ Load Data (Only Needed for Input Size)
X_clients = np.load('X_clients.npy')
input_size = X_clients.shape[1]

# ✅ Initialize the Global Model
global_model = GlobalModel(input_size)
global_parameters = [val.cpu().detach().numpy() for val in global_model.parameters()]

# ✅ Custom FedProx Strategy with Accuracy Calculation
class CustomFedProx(fl.server.strategy.FedAvg):
    def __init__(self, initial_parameters):
        super().__init__()
        self.initial_parameters = initial_parameters

    def initialize_parameters(self, client_manager):
        return ndarrays_to_parameters(self.initial_parameters)

    def aggregate_evaluate(
        self,
        rnd: int,
        results: List[Tuple[Parameters, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[float, dict]:
        """Aggregate evaluation results and print global accuracy."""
        
        if not results:
            print(f"⚠️ Round {rnd}: No evaluation results received!")
            return 0.0, {"accuracy": 0.0}  # ✅ Return a valid tuple (loss, metrics)

        accuracies = [r.metrics["accuracy"] for _, r in results]
        global_accuracy = sum(accuracies) / len(accuracies)

        print(f"📊 Round {rnd}: Global Model Accuracy: {global_accuracy:.4f}")

        return 0.0, {"accuracy": global_accuracy}  # ✅ Return (loss, metrics)

# ✅ Initialize Strategy with Global Model Parameters
strategy = CustomFedProx(initial_parameters=global_parameters)

# ✅ Start Federated Learning Server (Runs for 30 Rounds)
fl.server.start_server(
    server_address="0.0.0.0:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=10)  # Adjust the number of rounds
)

# ✅ Evaluate the Final Global Model
global_model.eval()
print("🎯 Final Global Model Evaluation Completed!")


	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=10, no round_timeout
INFO :      Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 0 results and 2 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures

📊 Round 1: Global Model Accuracy: 0.1634


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 2: Global Model Accuracy: 0.8482


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


...

📊 Round 8: Global Model Accuracy: 0.8565


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 10]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 9: Global Model Accuracy: 0.8510


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 366.59s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0
INFO :      		round 2: 0.0
INFO :      		round 3: 0.0
INFO :      		round 4: 0.0
INFO :      		round 5: 0.0
INFO :      		round 6: 0.0
INFO :      		round 7: 0.0
INFO :      		round 8: 0.0
INFO :      		round 9: 0.0
INFO :      		round 10: 0.0
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.163388007320862),
INFO :      	              (2, 0.848166489051909),
INFO :      	              (3, 0.8462059167810896),
INFO :      	              (4, 0.8517030308504188),
INFO :      	              (5, 0.8538673694478915),
INFO :      	              (6, 0.8435189486241279),
INFO :      	              (7, 0.846722255

📊 Round 10: Global Model Accuracy: 0.8600
🎯 Final Global Model Evaluation Completed!


version 3.2 secure agreagtion

In [ ]:
import torch
import torch.nn as nn
import flwr as fl
import numpy as np
from typing import List, Tuple
from flwr.common import Parameters, EvaluateRes, ndarrays_to_parameters

# ✅ Define the Global Model
class GlobalModel(nn.Module):
    def __init__(self, input_size):
        super(GlobalModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# ✅ Load Data (For Input Size)
X_clients = np.load('X_clients.npy')
input_size = X_clients.shape[1]

# ✅ Initialize the Global Model
global_model = GlobalModel(input_size)
global_parameters = [val.cpu().detach().numpy() for val in global_model.parameters()]

# ✅ Custom FedProx Strategy with Accuracy Calculation & Secure Aggregation
# Corrected CustomFedProx strategy
class CustomFedProx(fl.server.strategy.FedAvg):
    def __init__(self, initial_parameters):
        super().__init__(
            fraction_fit=1.0,
            fraction_evaluate=1.0,
            min_fit_clients=2,
            min_evaluate_clients=2,
            min_available_clients=2,
            # use_secure_aggregation REMOVED HERE
        )
        self.initial_parameters = initial_parameters

    def initialize_parameters(self, client_manager):
        return ndarrays_to_parameters(self.initial_parameters)

    def aggregate_evaluate(
        self,
        rnd: int,
        results: List[Tuple[Parameters, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[float, dict]:
        if not results:
            print(f"⚠️ Round {rnd}: No evaluation results received!")
            return 0.0, {"accuracy": 0.0}

        accuracies = [r.metrics["accuracy"] for _, r in results]
        global_accuracy = sum(accuracies) / len(accuracies)
        print(f"📊 Round {rnd}: Global Model Accuracy: {global_accuracy:.4f}")
        return 0.0, {"accuracy": global_accuracy}


# ✅ Initialize Strategy with Global Model Parameters
strategy = CustomFedProx(initial_parameters=global_parameters)

# ✅ Start Federated Learning Server (with Secure Aggregation enabled)
fl.server.start_server(
    server_address="0.0.0.0:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=10)
)

# ✅ Evaluate the Final Global Model
global_model.eval()
print("🎯 Final Global Model Evaluation Completed!")


	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


INFO :      Starting Flower server, config: num_rounds=10, no round_timeout
INFO :      Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 1: Global Model Accuracy: 0.6208


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 2: Global Model Accuracy: 0.6190


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 3: Global Model Accuracy: 0.6743


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


📊 Round 4: Global Model Accuracy: 0.6696


INFO :      aggregate_fit: received 1 results and 1 failures
